In [1]:
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers

import time 

### Data Processing

In [3]:
df = pd.read_csv('../data/num_data.csv')

In [4]:
dataset = df

In [5]:
dataset.shape

(420768, 16)

In [6]:
# Useful functions
def plot_predictions(test, predicted):
    plt.figure(figsize=(30, 15));

    plt.plot(test, color='red', alpha=0.5, label='Actual PM2.5 Concentration',)
    plt.plot(predicted, color='blue', alpha=0.5, label='Predicted PM2.5 Concentation')
    plt.title('PM2.5 Concentration Prediction')
    plt.xlabel('Time')
    plt.ylabel('PM2.5  Concentration')
    plt.legend()
    plt.show()
    

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    return rmse

In [7]:
data_size = dataset.shape[0]
train_size=int(data_size * 0.6)
test_size = 100
valid_size = data_size - train_size - test_size

test_next_day = [12, 24, 48]

In [8]:
training_set = dataset[:train_size].iloc[:,4:16].values
valid_set = dataset[train_size:train_size+valid_size].iloc[:,4:16].values
test_set = dataset[data_size-test_size:].iloc[:,4:16].values

In [9]:
y = dataset.iloc[:,4].values
y = y.reshape(-1,1)

y.shape

(420768, 1)

In [10]:
# Scaling the dataset
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
valid_set_scaled = sc.fit_transform(valid_set)
test_set_scaled = sc.fit_transform(test_set)

sc_y = MinMaxScaler(feature_range=(0,1))
y_scaled = sc_y.fit_transform(y)

In [11]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X_, y_ = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix-1:out_end_ix, 0]
        X_.append(seq_x)
        y_.append(seq_y)
    return np.array(X_), np.array(y_)

In [12]:
n_steps_in = 12
n_steps_out = 12
X_train, y_train = split_sequences(training_set_scaled, n_steps_in, n_steps_out)
X_valid, y_valid = split_sequences(valid_set_scaled, n_steps_in, n_steps_out)
X_test, y_test = split_sequences(test_set_scaled, n_steps_in, n_steps_out)

In [13]:
GRU_3 = Sequential()
LSTM_3 = Sequential()
GRU_4 =  Sequential()
LSTM_4 = Sequential()
LSTM_5 = Sequential()



GRU_3.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_3.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_3.add(GRU(units=50, activation='tanh'))
GRU_3.add(Dense(units=n_steps_out))

GRU_4.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_4.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_4.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_4.add(GRU(units=50, activation='tanh'))
GRU_4.add(Dense(units=n_steps_out))

LSTM_3.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_3.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_3.add(LSTM(units=50, activation='tanh'))
LSTM_3.add(Dense(units=n_steps_out))

LSTM_4.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_4.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_4.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_4.add(LSTM(units=50, activation='tanh'))
LSTM_4.add(Dense(units=n_steps_out))

LSTM_5.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_5.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_5.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_5.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_5.add(LSTM(units=50, activation='tanh'))
LSTM_5.add(Dense(units=n_steps_out))


# Compiling the RNNs
adam = optimizers.Adam(lr=0.01)

GRU_3.compile(optimizer=adam,loss='mean_squared_error')
GRU_4.compile(optimizer=adam,loss='mean_squared_error')
LSTM_3.compile(optimizer=adam,loss='mean_squared_error')
LSTM_4.compile(optimizer=adam,loss='mean_squared_error')
LSTM_5.compile(optimizer=adam,loss='mean_squared_error')



INFO:plaidml:Opening device "metal_intel(r)_iris(tm)_graphics_6100.0"


In [14]:
LSTM_GRU_LSTM = Sequential()
GRU_LSTM_GRU = Sequential()
LSTM_LSTM_GRU_GRU =  Sequential()
GRU_GRU_LSTM_LSTM = Sequential()
LSTM_GRU_LSTM_GRU = Sequential()
GRU_LSTM_GRU_LSTM = Sequential()


LSTM_GRU_LSTM.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_GRU_LSTM.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_GRU_LSTM.add(LSTM(units=50, activation='tanh'))
LSTM_GRU_LSTM.add(Dense(units=n_steps_out))

GRU_LSTM_GRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_LSTM_GRU.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_LSTM_GRU.add(GRU(units=50, activation='tanh'))
GRU_LSTM_GRU.add(Dense(units=n_steps_out))

LSTM_LSTM_GRU_GRU.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_LSTM_GRU_GRU.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_LSTM_GRU_GRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_LSTM_GRU_GRU.add(GRU(units=50, activation='tanh'))
LSTM_LSTM_GRU_GRU.add(Dense(units=n_steps_out))

GRU_GRU_LSTM_LSTM.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_GRU_LSTM_LSTM.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_GRU_LSTM_LSTM.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_GRU_LSTM_LSTM.add(LSTM(units=50, activation='tanh'))
GRU_GRU_LSTM_LSTM.add(Dense(units=n_steps_out))

LSTM_GRU_LSTM_GRU.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_GRU_LSTM_GRU.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_GRU_LSTM_GRU.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
LSTM_GRU_LSTM_GRU.add(GRU(units=50, activation='tanh'))
LSTM_GRU_LSTM_GRU.add(Dense(units=n_steps_out))

GRU_LSTM_GRU_LSTM.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_LSTM_GRU_LSTM.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_LSTM_GRU_LSTM.add(GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1],12), activation='tanh'))
GRU_LSTM_GRU_LSTM.add(LSTM(units=50, activation='tanh'))
GRU_LSTM_GRU_LSTM.add(Dense(units=n_steps_out))


# Compiling the RNNs
adam = optimizers.Adam(lr=0.01)

LSTM_GRU_LSTM.compile(optimizer=adam,loss='mean_squared_error')
GRU_LSTM_GRU.compile(optimizer=adam,loss='mean_squared_error')

LSTM_LSTM_GRU_GRU.compile(optimizer=adam,loss='mean_squared_error')
GRU_GRU_LSTM_LSTM.compile(optimizer=adam,loss='mean_squared_error')
LSTM_GRU_LSTM_GRU.compile(optimizer=adam,loss='mean_squared_error')
GRU_LSTM_GRU_LSTM.compile(optimizer=adam,loss='mean_squared_error')

In [15]:
RnnModelDict = {'LSTM_3': LSTM_3, 'GRU_3': GRU_3, 'LSTM_4': LSTM_4, 'GRU_4': GRU_4, 
                'LSTM_GRU_LSTM': LSTM_GRU_LSTM, 'GRU_LSTM_GRU': GRU_LSTM_GRU, 
                'LSTM_LSTM_GRU_GRU': LSTM_LSTM_GRU_GRU, 'GRU_GRU_LSTM_LSTM': GRU_GRU_LSTM_LSTM}

RnnModelDict = {'LSTM_5': LSTM_5}
X_test_24 = X_test[:24]
y_test_24 = y_test[:24]
rmse_df = pd.DataFrame()


In [16]:
for model in RnnModelDict:
    regressor = RnnModelDict[model]
    
    print('training start for', model)    
    start = time.process_time()
    regressor.fit(X_train,y_train,epochs=50,batch_size=32)
    train_time = round(time.process_time() - start, 2)
    
    print('results for training set')
    y_train_pred = regressor.predict(X_train)
#     plot_predictions(y_train,y_train_pred)
    train_rmse = return_rmse(y_train,y_train_pred)
    
    print('results for valid set')
    y_valid_pred = regressor.predict(X_valid)
#     plot_predictions(y_valid,y_valid_pred)
    valid_rmse = return_rmse(y_valid,y_valid_pred)
    
    
#     print('results for test set - 24 hours')
#     y_test_pred24 = regressor.predict(X_test_24)
#     plot_predictions(y_test_24,y_test_pred24)
#     test24_rmse = return_rmse(y_test_24,y_test_pred24)
    
    
    one_df = pd.DataFrame([[model, train_rmse, valid_rmse, train_time]], 
                          columns=['Model', 'train_rmse', 'valid_rmse', 'train_time'])
    rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../deep_rnn_12h.csv')


training start for LSTM_5
Epoch 1/50


INFO:plaidml:Analyzing Ops: 2647 of 6354 operations complete


252416/252438 [============================>.] - ETA: 0s - loss: 0.0024

INFO:plaidml:Analyzing Ops: 3442 of 6355 operations complete


252438/252438 [==============================] - 1337s 5ms/step - loss: 0.0024
Epoch 2/50
252438/252438 [==============================] - 1318s 5ms/step - loss: 0.0022
Epoch 3/50
252438/252438 [==============================] - 1324s 5ms/step - loss: 0.0021
Epoch 4/50
252438/252438 [==============================] - 1326s 5ms/step - loss: 0.0020
Epoch 5/50
252438/252438 [==============================] - 1327s 5ms/step - loss: 0.0020
Epoch 6/50
252438/252438 [==============================] - 1323s 5ms/step - loss: 0.0020
Epoch 7/50
252438/252438 [==============================] - 1320s 5ms/step - loss: 0.0019
Epoch 8/50
252438/252438 [==============================] - 1329s 5ms/step - loss: 0.0019
Epoch 9/50
252438/252438 [==============================] - 1323s 5ms/step - loss: 0.0019
Epoch 10/50
252438/252438 [==============================] - 1323s 5ms/step - loss: 0.0018
Epoch 11/50
252438/252438 [==============================] - 1327s 5ms/step - loss: 0.0019
Epoch 12/50
252438/